# ColorCamp

In [1]:
import colorcamp as cc

sky = cc.Hex('#15AAFF', name = 'sky')
sky

'#15AAFF'

## Working with colors and color groups
All colors and color groups can be tagged with an additional `name`, `description`, and unstructured dictionary `metadata`.

### Colors
Color camp allows users to represent colors in common color spaces: Hex, RGB, HSL. Colors are extensions of the common native type, e.g. Hex == `str`, RGB == `tuple`.


In [2]:
mustard = cc.Hex('#FFAA15', name = 'mustard')
lime = cc.Hex('#15FFAA', name = 'lime')
pink = cc.Hex('#FF15AA', name = 'pink')

# default representation
print(mustard)
# accessing string methods
print(mustard.lower())
print(f"is mustard a string: {isinstance(mustard, str)}")
# notebook html representation
mustard


#FFAA15
#ffaa15
is mustard a string: True


'#FFAA15'

Colors can be converted between colorspaces

In [3]:
mustard_hsl = mustard.to_hsl()
mustard_rgb = mustard.to_rgb()
display(mustard_hsl, mustard_rgb)

(38.205128, 1.0, 0.541176)

(255, 170, 21)

Alpha can be adjusted on colors

In [4]:
# Note this returns a new color object
mustard_rgb.change_alpha(0.7)

(255, 170, 21, 0.7)

Colors have many useful attributes and methods

In [5]:
new_pink = cc.RGB(
    (255,21,170), 
    name = 'super_pink',
    description="An energizing pink",
    metadata={"sentiment": "excited"},
)

display(new_pink)

print(
    f'{new_pink.name} red value: {new_pink.red}\n',
    f'{new_pink.name} hex string: {new_pink.hex}\n',
    f'{new_pink.name} css: {new_pink.css()}\n',
    f'{new_pink.name} alpha: {new_pink.alpha}\n',
    f'{new_pink.name} info: {new_pink.info()}\n',

)

(255, 21, 170)

super_pink red value: 255
 super_pink hex string: #FF15AA
 super_pink css: rgb(255, 21, 170)
 super_pink alpha: None
 super_pink info: {'name': 'super_pink', 'description': 'An energizing pink', 'metadata': {'sentiment': 'excited'}}



Colors can be added together to blend them together. The resulting color is casted as the left most color. They can also be tested in equalities

In [6]:
color_1 = new_pink + mustard
color_2 = mustard + new_pink

display(color_1, color_2)

print(f"These colors are the same: {new_pink == pink}")

(255, 96, 96)

'#FF6060'

These colors are the same: True


### Color Groups
Color groups are different ways of representing of collections of colors and include: Palettes, Scales, and Maps

### Palettes
Color palettes are ordered tuples of colors, useful for categorical values, brand colors, mood boards, etc.

In [7]:
brand_colors = cc.Palette(
    colors = [
        mustard,  
        lime, 
        sky,
        pink,
    ],
    name = 'bright_and_sunny'
)

# Palettes are extended tuples
print(f'Palettes are tuples: {isinstance(brand_colors, tuple)}')
# standard representation
print(brand_colors)
# html representation
brand_colors

Palettes are tuples: True
Palette('#FFAA15', '#15FFAA', '#15AAFF', '#FF15AA')


Palette('#FFAA15', '#15FFAA', '#15AAFF', '#FF15AA')

### Scales
Scales are similar to palettes, but they imply continuous data, gradients, and divergent data. Scales will infer linear interpolation between two colors. The relative distances between them can be adjusted with `stops`

In [8]:
# Scales can be created from palettes
brand_scale = cc.Scale(colors=brand_colors, name = 'BrandScale')
# Additionally you can use the built in method 
brand_scale.to_scale()
# Scales are extended tuples
print(f'Scales are tuples: {isinstance(brand_scale, tuple)}')

uneven_brand_scale = cc.Scale(colors=brand_colors, stops=[0, 0.75, 0.9, 1])
display(brand_scale,uneven_brand_scale)

Scales are tuples: True


Scale(('#FFAA15', 0.0), ('#15FFAA', 0.3333333333333333), ('#15AAFF', 0.6666666666666666), ('#FF15AA', 1))

Scale(('#FFAA15', 0), ('#15FFAA', 0.75), ('#15AAFF', 0.9), ('#FF15AA', 1))

### Maps
Color Maps are useful for explicitly assigning variables to colors. Effectively this a dictionary of colors, although they are not actually and instance of `dict`

**NOTE**: This is the only mutable color object

In [9]:
cmap = cc.Map(
    {color.name:color for color in brand_colors},
    name = 'BrandColorMap'
)

print(cmap)
cmap

Map{'mustard': '#FFAA15', 'lime': '#15FFAA', 'sky': '#15AAFF', 'pink': '#FF15AA'}


mustard,
lime,
sky,
pink,


### Saving and loading color objects
Colors and Color objects can be saved as JSON files which are easily shared between applications and other frameworks.

In [10]:
# Save several colors / color objects to the current directory

pink.dump_json('pink.json', overwrite=True)
cmap.dump_json('brand_map.json', overwrite=True)

# Loading objects back into memory

pink_reloaded = cc.Hex.load_json('pink.json')
display(pink_reloaded)

# The default color type can be changed at load or globally
## At load
cmap_reloaded = cc.Map.load_json('brand_map.json', color_space='HSL')
print(cmap_reloaded)

## Change global settings
cc.settings.default_color_type = 'RGB'
cmap_reloaded = cc.Map.load_json('brand_map.json')
print(cmap_reloaded)


'#FF15AA'

Map{'mustard': (38.205128, 1.0, 0.541176), 'lime': (158.205128, 1.0, 0.541176), 'sky': (201.794872, 1.0, 0.541176), 'pink': (321.794872, 1.0, 0.541176)}
Map{'mustard': '#FFAA15', 'lime': '#15FFAA', 'sky': '#15AAFF', 'pink': '#FF15AA'}


## Camps
Color camps are entire collections of colors and meant for larger organizational themes. The entire Camp can be saved and reloaded at once to quickly share colors and color objects between scripts, applications, and frameworks. All color objects in a camp must have a name attribute assigned.

The individual colors and color objects are saved in the following structure:

``` bash
CampName
├── colors
│   ├── pink
│   ├── ...
│   └── blue
├── palettes
│   ├── ...
│   └── ...
├── scales
│   ├── ...
│   └── ...
├── maps
│   ├── ...
│   └── ...
```

In [11]:
project_camp = cc.Camp(
    name = 'ExampleCamp',
    description = 'This is how we can organize colors to represent science more clearly!',
)

project_camp.colors.add(pink)
# Note: the `name` of this color is "super_pink"
project_camp.colors.add(new_pink) 
project_camp.colors.add(mustard)

project_camp.colors.super_pink

(255, 21, 170)

In [12]:
# Or you can add them in bulk
project_camp.add_objects([lime, sky, cmap, brand_colors, brand_scale])

display(project_camp.maps.BrandColorMap)
# They can also be accessed via the 'get item' syntax
project_camp.colors['pink']

mustard,
lime,
sky,
pink,


'#FF15AA'

Camps can be saved to be easily shared

In [13]:

project_camp.save(directory='.')
# reload camp
reloaded_project_camp = cc.Camp.load(name = 'ExampleCamp', directory='.')

ColorCamp ships with some common plotting color palettes
*  web colors
* colorbrewer
* carto
* seaborn/matplotlib

In [14]:
web_colors = cc.Camp.load('web_colors')
colorbrewer = cc.Camp.load('colorbrewer')
carto = cc.Camp.load('carto')
seaborn_camp = cc.Camp.load('seaborn')

carto.palettes.Agsunset

Palette('#4B2991', '#872CA2', '#C0369D', '#EA4F88', '#FA7876', '#F6A97A', '#EDD9A3')

ColorCamp will look for camps by default in the current working directory, and the installed package location. You can add or remove default search paths by changing `cc.settings.camp_paths`. All named paths can be displayed:

In [15]:
found_camps = cc.Camp.find()

for directory, found_names in found_camps.items():
    print(directory.split('/')[-1],found_names)


data ['carto', 'colorbrewer', 'crayons', 'seaborn', 'web_colors']
examples ['ExampleCamp']


Camps can also be made into html reports

In [16]:
from IPython.display import HTML

cc.report(project_camp)
